In [1]:
import cyecca.util
from cyecca import lie
import casadi as ca
import numpy as np
import cyecca

In [2]:
a = ca.SX.sym("a", 3)
omega = ca.SX.sym("omega", 3)

std_vel = ca.SX.sym("std_vel")
std_acc = ca.SX.sym("std_acc")
std_rot = ca.SX.sym("std_rot")

W = ca.SX.sym("W", ca.Sparsity.lower(9))
nu = lie.se23.elem(ca.vertcat(a, ca.SX.zeros(3, 1), omega))

Q = ca.SX.sym("Q", ca.Sparsity.diag(9))
Q = ca.diagcat(
    std_vel**2,
    std_vel**2,
    std_vel**2,
    std_acc**2,
    std_acc**2,
    std_acc**2,
    std_rot**2,
    std_rot**2,
    std_rot**2,
)
F = nu.ad()
B = ca.SX.eye(9)
W_dot_lt = ca.tril(cyecca.util.sqrt_covariance_predict(W, F, Q))
f_W_dot_lt = ca.Function(
    "f_cov_predict",
    [W, a, omega, std_vel, std_acc, std_rot],
    [W_dot_lt],
    ["W", "a", "omega", "std_vel", "std_acc", "std_rot"],
    ["W_dot_lt"],
)

dt = ca.SX.sym("dt")
W1 = cyecca.util.rk4(
    lambda t, y: f_W_dot_lt(W, a, omega, std_vel, std_acc, std_rot), 0, W, dt
)

In [3]:
Xh = lie.SE23Mrp.elem(ca.SX.sym("Xh", 9))
y_mag = lie.r3.elem(ca.SX.sym("y_mag", 3))
B_w = lie.r3.elem(ca.SX.sym("B_w", 3))


y_mag_hat = Xh.R.inverse() @ B_w
e_mag = y_mag - y_mag_hat

lie.SO3Mrp.from_Euler(lie.SO3EulerB321.elem(ca.vertcat(1, 0, 0)))

SO3MrpLieGroup: SX(@1=0, [@1, @1, 0.255342])

In [4]:
mag_incl = ca.SX.sym("mag_incl")
mag_decl = ca.SX.sym("mag_decl")

# using ENU
east_hat = lie.r3.elem(ca.vertcat(1, 0, 0))
north_hat = lie.r3.elem(ca.vertcat(0, 1, 0))
up_hat = lie.r3.elem(ca.vertcat(0, 0, 1))

b_w = lie.SO3EulerB321.elem(ca.vertcat(-mag_decl, 0, -mag_incl)) @ north_hat
cyecca.symbolic.casadi_to_sympy(b_w.param)

Matrix([
[sin(mag_decl)*cos(mag_incl)],
[cos(mag_decl)*cos(mag_incl)],
[             -sin(mag_incl)]])

In [5]:
y_mag_w = Xh.R.inverse() @ y_mag
psi_m = ca.atan2(y_mag_w.param[0], y_mag_w.param[1])
psi_m

SX(@1=1, @2=8, @3=((sq(Xh_6)+sq(Xh_7))+sq(Xh_8)), @4=sq((@1+@3)), @5=(4*(@1-@3)), atan2(((((@1-((@2*(sq(Xh_8)+sq(Xh_7)))/@4))*y_mag_0)+((((@2*(Xh_6*Xh_7))+(@5*Xh_8))/@4)*y_mag_1))+((((@2*(Xh_6*Xh_8))-(@5*Xh_7))/@4)*y_mag_2)),((((((@2*(Xh_7*Xh_6))-(@5*Xh_8))/@4)*y_mag_0)+((@1-((@2*(sq(Xh_8)+sq(Xh_6)))/@4))*y_mag_1))+((((@2*(Xh_7*Xh_8))+(@5*Xh_6))/@4)*y_mag_2))))

In [6]:
psi_hat = ca.atan2(b_w.param[0], b_w.param[1])
import sympy

sympy.simplify(
    cyecca.symbolic.casadi_to_sympy(psi_hat)
)  # just mag_decl, but should use this to figure out uncertainty?

atan2(sin(mag_decl)*cos(mag_incl), cos(mag_decl)*cos(mag_incl))

In [7]:
psi_m - mag_decl

SX(@1=1, @2=8, @3=((sq(Xh_6)+sq(Xh_7))+sq(Xh_8)), @4=sq((@1+@3)), @5=(4*(@1-@3)), (atan2(((((@1-((@2*(sq(Xh_8)+sq(Xh_7)))/@4))*y_mag_0)+((((@2*(Xh_6*Xh_7))+(@5*Xh_8))/@4)*y_mag_1))+((((@2*(Xh_6*Xh_8))-(@5*Xh_7))/@4)*y_mag_2)),((((((@2*(Xh_7*Xh_6))-(@5*Xh_8))/@4)*y_mag_0)+((@1-((@2*(sq(Xh_8)+sq(Xh_6)))/@4))*y_mag_1))+((((@2*(Xh_7*Xh_8))+(@5*Xh_6))/@4)*y_mag_2)))-mag_decl))

In [8]:
crz = lie.SO3EulerB321.elem(ca.vertcat(psi_m - mag_decl, 0, 0)).log().param[2]
f_crz = ca.Function("f_crz", [Xh.param, y_mag.param, mag_decl], [crz])
f_crz([0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0], 0)

DM(0)

In [9]:
crz

SX(@1=2, @2=0, @3=1, @4=8, @5=((sq(Xh_6)+sq(Xh_7))+sq(Xh_8)), @6=sq((@3+@5)), @7=4, @8=(@7*(@3-@5)), @9=(atan2(((((@3-((@4*(sq(Xh_8)+sq(Xh_7)))/@6))*y_mag_0)+((((@4*(Xh_6*Xh_7))+(@8*Xh_8))/@6)*y_mag_1))+((((@4*(Xh_6*Xh_8))-(@8*Xh_7))/@6)*y_mag_2)),((((((@4*(Xh_7*Xh_6))-(@8*Xh_8))/@6)*y_mag_0)+((@3-((@4*(sq(Xh_8)+sq(Xh_6)))/@6))*y_mag_1))+((((@4*(Xh_7*Xh_8))+(@8*Xh_6))/@6)*y_mag_2)))-mag_decl), @10=cos(@9), @11=(@2<((@10+@10)+@3)), @12=(!@11), @13=(@2&&(@3<@10)), @14=(!@13), @15=(@3<@10), @16=nan, @17=(@12?(@13?(@12?(@14?(@15?@16:0):0):0):0):0), @18=0.5, @19=(@18*sqrt((((@3+@10)+@10)+@3))), @20=(!@15), @21=sin(@9), @22=(@18*sqrt((((@3-@10)-@10)+@3))), @23=((@11?@19:0)+(@12?((@13?@16:0)+(@14?((@15?@16:0)+(@20?((@21+@21)/(@7*@22)):0)):0)):0)), @24=((@11?((@21+@21)/(@7*@19)):0)+(@12?((@13?@16:0)+(@14?((@15?@16:0)+(@20?@22:0)):0)):0)), @25=(@23*@24), @26=sq(@23), @27=(@12?(@14?(@15?(@12?(@14?(@15?@16:0):0):0):0):0):0), @28=(@12?(@13?(@12?(@13?@16:0):0):0):0), @29=sq(@24), @30=(((((((@26+@27

In [10]:
f_crz(Xh.param, y_mag.param, mag_decl)

SX(@1=2, @2=0, @3=1, @4=8, @5=((sq(Xh_6)+sq(Xh_7))+sq(Xh_8)), @6=sq((@3+@5)), @7=4, @8=(@7*(@3-@5)), @9=(atan2(((((@3-((@4*(sq(Xh_8)+sq(Xh_7)))/@6))*y_mag_0)+((((@4*(Xh_6*Xh_7))+(@8*Xh_8))/@6)*y_mag_1))+((((@4*(Xh_6*Xh_8))-(@8*Xh_7))/@6)*y_mag_2)),((((((@4*(Xh_7*Xh_6))-(@8*Xh_8))/@6)*y_mag_0)+((@3-((@4*(sq(Xh_8)+sq(Xh_6)))/@6))*y_mag_1))+((((@4*(Xh_7*Xh_8))+(@8*Xh_6))/@6)*y_mag_2)))-mag_decl), @10=cos(@9), @11=(@2<((@10+@10)+@3)), @12=(!@11), @13=(@2&&(@3<@10)), @14=(!@13), @15=(@3<@10), @16=nan, @17=(@12?(@13?(@12?(@14?(@15?@16:0):0):0):0):0), @18=0.5, @19=(@18*sqrt((((@3+@10)+@10)+@3))), @20=(!@15), @21=sin(@9), @22=(@18*sqrt((((@3-@10)-@10)+@3))), @23=((@11?@19:0)+(@12?((@13?@16:0)+(@14?((@15?@16:0)+(@20?((@21+@21)/(@7*@22)):0)):0)):0)), @24=((@11?((@21+@21)/(@7*@19)):0)+(@12?((@13?@16:0)+(@14?((@15?@16:0)+(@20?@22:0)):0)):0)), @25=(@23*@24), @26=sq(@23), @27=(@12?(@14?(@15?(@12?(@14?(@15?@16:0):0):0):0):0):0), @28=(@12?(@13?(@12?(@13?@16:0):0):0):0), @29=sq(@24), @30=(((((((@26+@27

In [11]:
psi_h = lie.SO3EulerB321.from_Mrp(Xh.R).param[0]
psi_h

SX(@1=8, @2=1, @3=((sq(Xh_6)+sq(Xh_7))+sq(Xh_8)), @4=(4*(@2-@3)), @5=sq((@2+@3)), @6=asin((-(((@1*(Xh_8*Xh_6))-(@4*Xh_7))/@5))), @7=1.5708, @8=0.001, @9=(fabs((@6-@7))<@8), @10=(((@1*(Xh_7*Xh_8))-(@4*Xh_6))/@5), @11=(((@1*(Xh_6*Xh_8))+(@4*Xh_7))/@5), @12=(fabs((@6+@7))<@8), ((@9?atan2(@10,@11):0)+((!@9)?((@12?atan2((-@10),(-@11)):0)+((!@12)?atan2((((@1*(Xh_7*Xh_6))+(@4*Xh_8))/@5),(@2-((@1*(sq(Xh_8)+sq(Xh_7)))/@5))):0)):0)))

In [12]:
lie.SO3Mrp.from_Euler(lie.SO3EulerB321.elem(ca.vertcat(1, 0, 0)))

SO3MrpLieGroup: SX(@1=0, [@1, @1, 0.255342])

In [13]:
f_F = ca.Function("F", [a, omega, dt], [(nu * dt).exp(lie.SE23Mrp).Ad()])
F = f_F(a, omega, dt)

x_00 = ca.SX.sym("x_00", 9, 1)

x_10 = F @ x_00

np.array(f_F([1, 2, 3], [1, 2, 3], 0.1))

array([[ 0.9357548 , -0.28316496,  0.21019171,  0.        ,  0.        ,
         0.        , -0.12698783, -0.25970727,  0.21546746],
       [ 0.30293271,  0.95058062, -0.06803132,  0.        ,  0.        ,
         0.        ,  0.29878045, -0.09768295, -0.03447152],
       [-0.18054008,  0.12733457,  0.97529031,  0.        ,  0.        ,
         0.        , -0.15685769,  0.15169105, -0.04884147],
       [ 0.        ,  0.        ,  0.        ,  0.9357548 , -0.28316496,
         0.21019171,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.30293271,  0.95058062,
        -0.06803132,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , -0.18054008,  0.12733457,
         0.97529031,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.9357548 , -0.28316496,  0.21019171],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0

In [14]:
f_F(a, omega, dt)

SX(@1=1, @2=8, @3=(dt*omega_1), @4=(dt*omega_0), @5=(dt*omega_2), @6=((sq(@4)+sq(@3))+sq(@5)), @7=0.001, @8=(fabs(@6)<@7), @9=((@8?(((((0.166667+(-0.00833333*@6))+(-2.75573e-06*(@6*sq(@6))))+(-1.6059e-10*(@6*sq(sq(@6)))))+(0.000198413*sq(@6)))+(2.50521e-08*sq(sq(@6)))):0)+((!@8)?(pow(@6,-1.5)*(sqrt(@6)-sin(sqrt(@6)))):0)), @10=(@9*@4), @11=(fabs(@6)<@7), @12=0.5, @13=((@11?(((((@12+(-0.0416667*@6))+(-2.48016e-05*(@6*sq(@6))))+(-2.08768e-09*(@6*sq(sq(@6)))))+(0.00138889*sq(@6)))+(2.75573e-07*sq(sq(@6)))):0)+((!@11)?((@1-cos(sqrt(@6)))/@6):0)), @14=((@10*@5)-(@13*@3)), @15=(@9*@4), @16=((@13*@5)+(@15*@3)), @17=((@3*@14)-(@5*@16)), @18=(@1+@17), @19=(@9*@3), @20=((@19*@4)-(@13*@5)), @21=(@9*@3), @22=((@13*@4)+(@21*@5)), @23=(@1+((@5*@20)-(@4*@22))), @24=(@9*@5), @25=((@24*@3)-(@13*@4)), @26=(@9*@5), @27=((@13*@3)+(@26*@4)), @28=(@1+((@4*@25)-(@3*@27))), @29=(0<((@18+@23)+@28)), @30=(@1-((@24*@5)+(@15*@4))), @31=((@4*@30)-(@3*@20)), @32=(@1-((@21*@3)+(@10*@4))), @33=((@5*@27)-(@4*@32)), @3

# Covariance Prediction

In [15]:
def derive_cov_sqrt_predict():
    a = ca.SX.sym("a", 3)
    omega = ca.SX.sym("omega", 3)

    std_vel = ca.SX.sym("std_vel")
    std_acc = ca.SX.sym("std_acc")
    std_rot = ca.SX.sym("std_rot")

    W = ca.SX.sym("W", ca.Sparsity.lower(9))
    nu = lie.se23.elem(ca.vertcat(a, ca.SX.zeros(3, 1), omega))

    Q = ca.SX.sym("Q", ca.Sparsity.diag(9))
    Q = ca.diagcat(
        std_vel**2,
        std_vel**2,
        std_vel**2,
        std_acc**2,
        std_acc**2,
        std_acc**2,
        std_rot**2,
        std_rot**2,
        std_rot**2,
    )
    F = nu.ad()
    B = ca.SX.eye(9)
    W_dot_lt = ca.tril(cyecca.util.sqrt_covariance_predict(W, F, Q))
    f_W_dot_lt = ca.Function(
        "f_cov_predict",
        [W, a, omega, std_vel, std_acc, std_rot],
        [W_dot_lt],
        ["W", "a", "omega", "std_vel", "std_acc", "std_rot"],
        ["W_dot_lt"],
    )

    dt = ca.SX.sym("dt")
    W1 = cyecca.util.rk4(
        lambda t, y: f_W_dot_lt(W, a, omega, std_vel, std_acc, std_rot), 0, W, dt
    )

    return ca.Function(
        "cov_sqrt_predict",
        [W, a, omega, std_vel, std_acc, std_rot, dt],
        [W1],
        ["W", "a", "omega", "std_vel", "std_acc", "std_rot", "dt"],
        ["W1"],
    )


f_cov_sqrt_predict = derive_cov_sqrt_predict()

In [16]:
def do_cov_sim():
    np.set_printoptions(precision=5, linewidth=200, suppress=True)
    W = 1 * np.eye(9)
    a = np.array([1, 2, 3])
    omega = np.array([1, 2, 3])
    dt = 0.01
    std_vel = 1
    std_acc = 2
    std_rot = 3
    for i in range(100):
        W = np.array(f_cov_sqrt_predict(W, a, omega, std_vel, std_acc, std_rot, dt))

    return W


do_cov_sim()

array([[ 7.39164,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [-1.08925,  6.42452,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [-1.63906, -4.05501,  1.74906,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     ,  2.23969,  0.     ,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     ,  0.     ,  2.23969,  0.     ,  0.     ,  0.     ,  0.     ],
       [ 0.     ,  0.     ,  0.     , -0.     ,  0.     ,  2.23969,  0.     ,  0.     ,  0.     ],
       [ 0.00022,  2.60619, -0.33359,  0.     ,  0.     ,  0.     ,  1.79103,  0.     ,  0.     ],
       [-2.26587, -0.38356,  0.17048,  0.     ,  0.     ,  0.     ,  0.58833,  2.10766,  0.     ],
       [ 1.50902, -0.6096 , -0.00283,  0.     ,  0.     ,  0.     ,  0.88303,  1.25367,  2.25231]])